<a href="https://colab.research.google.com/github/Adarsh3589/AMS5659-15-5PH-DATA-/blob/main/15-5%20PH%20DATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
import requests
from io import BytesIO

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm, anderson

from google.colab import files
from IPython.display import display
import ipywidgets as widgets


# Step 1: Download the Excel file from GitHub
url = "https://raw.githubusercontent.com/Adarsh3589/AMS5659-15-5PH-DATA-/ae94dc3642843476b37acae8a9a468726a753ae4/Data%2015-5%20PH.xlsx"

response = requests.get(url)
response.raise_for_status()
excel_data = BytesIO(response.content)

# Step 2: Load the second sheet
df = pd.read_excel(excel_data, sheet_name=0, engine='openpyxl')

# Create output directory
os.makedirs("output", exist_ok=True)

In [ ]:
# Cell 2: Drop rows with missing 'Condition' or 'HT Type'
df.dropna(subset=['Condition', 'HT Type'], inplace=True)
print("Rows after dropping NA in Condition or HT Type:", df.shape)


In [ ]:
import ipywidgets as widgets
from IPython.display import display

# === Step 1: Prepare dropdowns ===
condition_dropdown = widgets.Dropdown(
    options=sorted(df['Condition'].dropna().unique()),
    description="Condition:",
    value="H1025" if "H1025" in df['Condition'].values else None
)

size_options = ["All"] + sorted(df['Size'].dropna().unique())
size_multiselect = widgets.SelectMultiple(
    options=size_options,
    value=("All",),
    description="Size:"
)

# === Step 2: Container for filtered data ===
filtered_data = {"df": None}  # Shared container across cells

# === Step 3: Filtering logic ===
def filter_df(condition_value, size_values):
    filtered = df[df['Condition'] == condition_value]
    filtered = filtered.sort_values(by=['ESR', 'Melting Date'])
    if "All" not in size_values:
        filtered = filtered[filtered['Size'].isin(size_values)]
    return filtered

# === Step 4: Callback to update and store filtered data ===
def on_filter_change(change=None):
    result = filter_df(condition_dropdown.value, size_multiselect.value)
    filtered_data["df"] = result
    print(f"Filtered shape: {result.shape}")
    display(result.head())

# === Step 5: Bind widget event ===
condition_dropdown.observe(on_filter_change, names='value')
size_multiselect.observe(on_filter_change, names='value')

# === Step 6: Display widgets ===
display(widgets.VBox([condition_dropdown, size_multiselect]))

# Run initial display
on_filter_change()


In [ ]:
filtered